In [2]:
import pandas as pd
import numpy as np

# demographic information of census blocks
demos_df = pd.read_csv('/content/drive/MyDrive/Autoencoders/data_directory/nhgis0010_ds262_20225_blck_grp.csv')
demos_df.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AQRUM034,AQRUM035,AQRUM036,AQRUM037,AQRUM038,AQRUM039,AQRUM040,AQRUM041,AQRUM042,AQRUM043
0,G38000109656001,2018-2022,ND,NaN,NaN,North Dakota,38,Adams County,1,NaN,...,7,10,12,10,19,12,12,10,10,9
1,G38000109656002,2018-2022,ND,NaN,NaN,North Dakota,38,Adams County,1,NaN,...,20,12,32,27,8,19,15,15,12,11
2,G38000309679001,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,3,18,6,33,6,28,8,9,21,10
3,G38000309679002,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,28,53,26,23,18,6,7,9,29,15
4,G38000309680001,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,4,9,10,16,14,4,18,5,9,7


In [ ]:
demos_df.describe()

,REGIONA,DIVISIONA,STATEA,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,...,AQRUM034,AQRUM035,AQRUM036,AQRUM037,AQRUM038,AQRUM039,AQRUM040,AQRUM041,AQRUM042,AQRUM043
count,0.0,0.0,632.0,632.000000,0.0,0.0,632.000000,632.000000,0.0,0.0,...,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000
mean,NaN,NaN,38.0,49.778481,NaN,NaN,467572.398734,2.064873,NaN,NaN,...,25.060127,25.248418,24.810127,23.704114,23.018987,20.789557,28.905063,23.077532,26.925633,22.169304
std,NaN,NaN,0.0,34.206815,NaN,NaN,473901.779878,1.041409,NaN,NaN,...,30.136607,28.584847,26.695952,25.274270,25.741854,22.342494,31.338404,25.263907,31.378612,25.540648
min,NaN,NaN,38.0,1.000000,NaN,NaN,100.000000,1.000000,NaN,NaN,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,NaN,NaN,38.0,17.000000,NaN,NaN,10600.750000,1.000000,NaN,NaN,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
50%,NaN,NaN,38.0,35.000000,NaN,NaN,40508.000000,2.000000,NaN,NaN,...,14.000000,14.000000,15.000000,14.000000,13.000000,10.000000,18.000000,13.000000,14.000000,10.000000
75%,NaN,NaN,38.0,85.500000,NaN,NaN,961700.000000,3.000000,NaN,NaN,...,28.250000,30.000000,30.000000,27.250000,27.000000,24.000000,34.250000,27.000000,32.000000,26.000000
max,NaN,NaN,38.0,105.000000,NaN,NaN,974200.000000,6.000000,NaN,NaN,...,407.000000,291.000000,311.000000,204.000000,230.000000,168.000000,297.000000,211.000000,324.000000,192.000000


In [ ]:
demos_df.isnull().sum()

GISJOIN        0
YEAR           0
STUSAB         0
REGIONA      632
DIVISIONA    632
            ... 
AQRUM039       0
AQRUM040       0
AQRUM041       0
AQRUM042       0
AQRUM043       0
Length: 539, dtype: int64

In [ ]:
# prompt: check the number of items

print(f"Number of items: {demos_df.shape[0]}")


Number of items: 632


In [ ]:
print(demos_df.dtypes)

GISJOIN       object
YEAR          object
STUSAB        object
REGIONA      float64
DIVISIONA    float64
              ...   
AQRUM039       int64
AQRUM040       int64
AQRUM041       int64
AQRUM042       int64
AQRUM043       int64
Length: 539, dtype: object


In [3]:
# Define the columns for each age category
age_categories = {
    '25_34yrs': ['AQM4E011', 'AQM4E012', 'AQM4E035', 'AQM4E036'],
    '35_50yrs': ['AQM4E013', 'AQM4E014', 'AQM4E015', 'AQM4E037', 'AQM4E038', 'AQM4E039'],
    'over65yrs': ['AQM4E020', 'AQM4E021', 'AQM4E022', 'AQM4E023', 'AQM4E024', 'AQM4E045', 'AQM4E046', 'AQM4E047', 'AQM4E048', 'AQM4E049']
}

# Calculate the percentages and totals
for category, columns in age_categories.items():
    # Calculate the total count for each age category
    demos_df[category + '_count'] = demos_df[columns].sum(axis=1)
    # Calculate the percentage for each age category
    demos_df['pct' + category] = (demos_df[category + '_count'] / demos_df['AQM4E001']) * 100

# Display the first few rows of the dataframe to check the results
demos_df[['25_34yrs_count', 'pct25_34yrs', '35_50yrs_count', 'pct35_50yrs', 'over65yrs_count', 'pctover65yrs']].head()


,25_34yrs_count,pct25_34yrs,35_50yrs_count,pct35_50yrs,over65yrs_count,pctover65yrs
0,47,6.509695,92,12.742382,172,23.822715
1,148,10.081744,218,14.850136,357,24.318801
2,94,10.633484,74,8.371041,233,26.357466
3,58,6.847698,196,23.140496,146,17.237308
4,58,8.529412,114,16.764706,119,17.500000


In [4]:
# Define the columns for each racial category
racial_categories = {
    'White_alone': ['AQNGE002'],
    'Black_or_African_American_alone': ['AQNGE003'],
    'Other_Race_alone': ['AQNGE004', 'AQNGE005', 'AQNGE006', 'AQNGE007', 'AQNGE008', 'AQNGE009', 'AQNGE010']
}

# Create new columns for each racial category count and percentage
for category, columns in racial_categories.items():
    demos_df[category + '_count'] = demos_df[columns].sum(axis=1)
    demos_df[category + '_pct'] = demos_df[category + '_count'] / demos_df['AQNGE001'] * 100

# Calculate the count for nonwhite category
demos_df['nonwhite_count'] = demos_df[['Black_or_African_American_alone_count', 'Other_Race_alone_count']].sum(axis=1)

# Calculate the percentage for nonwhite category
demos_df['pct_nonwhite'] = demos_df['nonwhite_count'] / demos_df['AQNGE001'] * 100

# Renaming the columns for clarity
demos_df.rename(columns={
    'White_alone_pct': 'pctwhite_alone',
    'Black_or_African_American_alone_pct': 'pctblack_alone',
    'White_alone_count': 'white_alone_count',
    'Black_or_African_American_alone_count': 'black_alone_count',
    'Other_Race_alone_count': 'other_race_count',
    'nonwhite_count': 'nonwhite_count'
}, inplace=True)

# Display the first few rows of the dataframe to check the results
demos_df[['pctwhite_alone', 'pctblack_alone', 'pct_nonwhite', 'white_alone_count', 'nonwhite_count', 'black_alone_count']].head()


,pctwhite_alone,pctblack_alone,pct_nonwhite,white_alone_count,nonwhite_count,black_alone_count
0,97.368421,0.969529,2.631579,703,19,7
1,94.754768,0.272480,7.970027,1391,117,4
2,97.511312,0.000000,4.977376,862,44,0
3,96.458087,0.000000,3.896104,817,33,0
4,96.617647,0.000000,5.735294,657,39,0


In [5]:
# Define columns for educational attainment categories with a degree
edu_columns = ['AQPKE021', 'AQPKE022', 'AQPKE023', 'AQPKE024', 'AQPKE025']

# Calculate total population 25 years and over
total_population_25_over = demos_df['AQPKE001']

# Calculate total population with at least a bachelor's degree
population_with_college_degree = demos_df[edu_columns].sum(axis=1)

# Calculate percentage with a college degree
demos_df['pct_col_degree'] = (population_with_college_degree / total_population_25_over) * 100

# Calculate count of individuals with at least a bachelor's degree
demos_df['col_degree_count'] = population_with_college_degree

# Display the first few rows of the DataFrame to check the results
demos_df[['pct_col_degree', 'col_degree_count']].head()


,pct_col_degree,col_degree_count
0,32.388664,160
1,30.961183,335
2,38.601824,254
3,39.935588,248
4,40.000000,204


In [6]:
# Define columns for travel time to work
travel_time_cols = ['AQN2M002', 'AQN2M003', 'AQN2M004', 'AQN2M005', 'AQN2M006',
                    'AQN2M007', 'AQN2M008', 'AQN2M009', 'AQN2M010']

# Define median values corresponding to each time category
tt_medians = [7.5, 12.5, 17, 22, 27, 32, 39.5, 52, 74.5]

# Calculate total population
total_population = demos_df['AQN2M001']

# Calculate average travel time to work
demos_df['avg_tt_to_work'] = np.dot(demos_df[travel_time_cols].values, tt_medians) / total_population

# Displaying the relevant columns
print(demos_df[['avg_tt_to_work']].head())


   avg_tt_to_work
0       80.490741
1       62.447917
2       94.635294
3       68.983146
4       64.968354


In [7]:
total_pt_commute = demos_df['AQN2M061']
total_commute = demos_df['AQN2M001']
demos_df['PTcommute_count'] = total_pt_commute

# Calculate percentage of population using public transportation to commute
demos_df['pctPTcommute'] = (total_pt_commute / total_commute) * 100

# Display the percentage and count of population using public transportation to commute
demos_df[['pctPTcommute', 'PTcommute_count']].head()

,pctPTcommute,PTcommute_count
0,18.518519,10
1,10.416667,10
2,11.764706,10
3,11.235955,10
4,12.658228,10


In [8]:
# Define income columns for males and females
income_male_cols = [
    'AQRUM003', 'AQRUM004', 'AQRUM005', 'AQRUM006', 'AQRUM007',
    'AQRUM008', 'AQRUM009', 'AQRUM010', 'AQRUM011', 'AQRUM012',
    'AQRUM013', 'AQRUM014', 'AQRUM015', 'AQRUM016', 'AQRUM017',
    'AQRUM018', 'AQRUM019', 'AQRUM020', 'AQRUM021', 'AQRUM022'
]

income_female_cols = [
    'AQRUM024', 'AQRUM025', 'AQRUM026', 'AQRUM027', 'AQRUM028',
    'AQRUM029', 'AQRUM030', 'AQRUM031', 'AQRUM032', 'AQRUM033',
    'AQRUM034', 'AQRUM035', 'AQRUM036', 'AQRUM037', 'AQRUM038',
    'AQRUM039', 'AQRUM040', 'AQRUM041', 'AQRUM042', 'AQRUM043'
]

# Define the median values for each income category
income_medians = [
    1250, 3750, 6250, 8750, 11250,
    13750, 16250, 18750, 21250, 23750,
    27500, 32500, 37500, 42500, 47500,
    52500, 59500, 69500, 87500, 100000
]

# Calculate the weighted sum of incomes for males
total_male_income = np.dot(demos_df[income_male_cols].values, income_medians)

# Calculate the weighted sum of incomes for females
total_female_income = np.dot(demos_df[income_female_cols].values, income_medians)

# Sum the male and female incomes to get the total income
total_income = total_male_income + total_female_income

# Calculate total population (using AQRUM001 which is the total population)
total_population = demos_df['AQRUM001']

# Calculate the per capita income
demos_df['inc_per_capita'] = total_income / total_population

# Display the first few rows of the relevant column
print(demos_df[['inc_per_capita']].head())


   inc_per_capita
0   215330.128205
1   230821.428571
2   208078.651685
3   232895.000000
4   202196.875000


In [9]:
demos_df['geoid'] = '38_'+demos_df['COUNTYA'].astype(str)+'_'+demos_df['TRACTA'].astype(str)
demos_df['geoid'].head()

0    38_1_965600
1    38_1_965600
2    38_3_967900
3    38_3_967900
4    38_3_968000
Name: geoid, dtype: object

In [10]:
# Selecting specific columns
new_df = demos_df[['COUNTYA', 'TRACTA', 'AQM4E001',
                   'pct25_34yrs', 'pct35_50yrs', 'pctover65yrs',
                   'pctwhite_alone', 'pct_nonwhite', 'pctblack_alone',
                   'pct_col_degree', 'pctPTcommute',
                   'white_alone_count', 'nonwhite_count', 'black_alone_count',
                   '25_34yrs_count','35_50yrs_count','over65yrs_count',
                   'col_degree_count', 'PTcommute_count',
                   'avg_tt_to_work', 'inc_per_capita', 'geoid']]


new_df = new_df.rename(columns = {'AQM4E001':'tot_population'})

# Display the first few rows of the new DataFrame
new_df.head()

,COUNTYA,TRACTA,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,pct_col_degree,...,nonwhite_count,black_alone_count,25_34yrs_count,35_50yrs_count,over65yrs_count,col_degree_count,PTcommute_count,avg_tt_to_work,inc_per_capita,geoid
0,1,965600,722,6.509695,12.742382,23.822715,97.368421,2.631579,0.969529,32.388664,...,19,7,47,92,172,160,10,80.490741,215330.128205,38_1_965600
1,1,965600,1468,10.081744,14.850136,24.318801,94.754768,7.970027,0.272480,30.961183,...,117,4,148,218,357,335,10,62.447917,230821.428571,38_1_965600
2,3,967900,884,10.633484,8.371041,26.357466,97.511312,4.977376,0.000000,38.601824,...,44,0,94,74,233,254,10,94.635294,208078.651685,38_3_967900
3,3,967900,847,6.847698,23.140496,17.237308,96.458087,3.896104,0.000000,39.935588,...,33,0,58,196,146,248,10,68.983146,232895.000000,38_3_967900
4,3,968000,680,8.529412,16.764706,17.500000,96.617647,5.735294,0.000000,40.000000,...,39,0,58,114,119,204,10,64.968354,202196.875000,38_3_968000


## Census Tract Area

In [11]:
import geopandas as gpd
census_area = gpd.read_file("/content/drive/MyDrive/Autoencoders/data_directory/gis_data/US_tract_2022.shp")

In [12]:
census_area['geometry'] = census_area['geometry'].to_crs('epsg:26916')
census_area['area'] = census_area['geometry'].area/1000000
census_area['geoid'] = census_area['STATEFP'].astype(str)+'_'+census_area['COUNTYFP'].str.lstrip('0')+'_'+census_area['TRACTCE'].str.lstrip('0')

In [13]:
new_df = new_df.merge(census_area[['geoid','area','INTPTLAT','INTPTLON']], on='geoid')

In [14]:
new_df.to_csv("demo_tract.csv", index=False)

In [ ]:
new_df.columns

Index(['COUNTYA', 'TRACTA', 'tot_population', 'pct25_34yrs', 'pct35_50yrs',
       'pctover65yrs', 'pctwhite_alone', 'pct_nonwhite', 'pctblack_alone',
       'pct_col_degree', 'pctPTcommute', 'white_alone_count', 'nonwhite_count',
       'black_alone_count', '25_34yrs_count', '35_50yrs_count',
       'over65yrs_count', 'col_degree_count', 'PTcommute_count',
       'avg_tt_to_work', 'inc_per_capita', 'geoid', 'area', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

In [15]:
new_df.head()

,COUNTYA,TRACTA,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,pct_col_degree,...,35_50yrs_count,over65yrs_count,col_degree_count,PTcommute_count,avg_tt_to_work,inc_per_capita,geoid,area,INTPTLAT,INTPTLON
0,1,965600,722,6.509695,12.742382,23.822715,97.368421,2.631579,0.969529,32.388664,...,92,172,160,10,80.490741,215330.128205,38_1_965600,2650.221230,+46.0968146,-102.5331983
1,1,965600,1468,10.081744,14.850136,24.318801,94.754768,7.970027,0.272480,30.961183,...,218,357,335,10,62.447917,230821.428571,38_1_965600,2650.221230,+46.0968146,-102.5331983
2,3,967900,884,10.633484,8.371041,26.357466,97.511312,4.977376,0.000000,38.601824,...,74,233,254,10,94.635294,208078.651685,38_3_967900,1643.882426,+46.9370110,-097.8551057
3,3,967900,847,6.847698,23.140496,17.237308,96.458087,3.896104,0.000000,39.935588,...,196,146,248,10,68.983146,232895.000000,38_3_967900,1643.882426,+46.9370110,-097.8551057
4,3,968000,680,8.529412,16.764706,17.500000,96.617647,5.735294,0.000000,40.000000,...,114,119,204,10,64.968354,202196.875000,38_3_968000,2318.138854,+46.9379265,-098.2386037
